# Design Pattern 12: Checkpoints

> Padrão usado em treinamento de modelos para salvar periodicamente o estado completo do modelo durante o processo de treinamento. Isso é feito para que modelos parcialmente treinados estejam disponíveis e possam ser usados como modelos finais (no caso de parada antecipada) ou como pontos de partida para treinamentos subsequentes (como em caso de falha da máquina ou ajuste fino).

### Bibliotecas

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

2024-06-24 17:47:38.035093: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-24 17:47:38.228822: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 17:47:39.622299: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 17:47:39.627954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 17:47:41.141391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Checkpoint

#### Dados

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

11490434/11490434 [==============================] - 6s 1us/step


#### Modelo

In [3]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#### Checkpoint: Steps per epoch

> Em vez de treinar o modelo por um número fixo de épocas podemos treiná-lo por um número específico de passos onde cada passo corresponde a uma atualização dos pesos do modelo com um lote de dados. Isso nos permite ter mais granularidade no controle do treinamento.

In [4]:
# Número total de passos de treinamento que serão realizados.
NUM_STEPS = 143000
# Tamanho de cada lote de dados usado durante o treinamento.
BATCH_SIZE = 100
# Quantos checkpoints serão salvos durante o treinamento.
NUM_CHECKPOINTS = 15

# Define o caminho onde os pesos do modelo serão salvos.
checkpoint_path = './checkpoints/model_checkpoint'
# Callback ModelCheckpoint que salva os pesos do modelo em checkpoint_path
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq=NUM_STEPS // NUM_CHECKPOINTS)


train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# Embaralha, agrupa os dados em lotes e repete o dataset indefinidamente. 
# A repetição é necessária para garantir que o modelo veja os dados continuamente durante o treinamento.
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).repeat()


eval_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
eval_dataset = eval_dataset.batch(BATCH_SIZE)


history = model.fit(train_dataset,
                    epochs=NUM_CHECKPOINTS, # Cada época virtual é definida pelo número de checkpoints
                    steps_per_epoch=NUM_STEPS // NUM_CHECKPOINTS, # Número de passos por época
                    validation_data=eval_dataset,
                    callbacks=[cp_callback])

test_loss, test_acc = model.evaluate(eval_dataset, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/15
9533/9533 [==============================] - 17s 2ms/step - loss: 0.0650 - accuracy: 0.9819 - val_loss: 0.0818 - val_accuracy: 0.9778
Epoch 2/15
9533/9533 [==============================] - 17s 2ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.0941 - val_accuracy: 0.9809
Epoch 3/15
9533/9533 [==============================] - 17s 2ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.1060 - val_accuracy: 0.9809
Epoch 4/15
9533/9533 [==============================] - 16s 2ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.1342 - val_accuracy: 0.9794
Epoch 5/15
9533/9533 [==============================] - 16s 2ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.1250 - val_accuracy: 0.9807
Epoch 6/15
9533/9533 [==============================] - 16s 2ms/step - loss: 7.9010e-04 - accuracy: 0.9998 - val_loss: 0.1390 - val_accuracy: 0.9800
Epoch 7/15
9533/9533 [==============================] - 16s 2ms/step - loss: 8.3202e-04 - accuracy: 0.9998 - val_loss: 0.1449 

#### Checkpoint: Virtual epochs

> O conceito de "Virtual epochs" visa manter constante o número total de exemplos de treinamento que o modelo vê, independentemente de quantos dados são adicionados ao longo do tempo. Isso é feito ajustando o número de passos por época com base no número total desejado de exemplos de treinamento.

In [5]:
NUM_TRAINING_EXAMPLES = 1000000 # Número total de exemplos de treinamento inicial
STOP_POINT = 14.3  # Ponto de parada virtual que indica quantas "épocas virtuais" são desejadas
BATCH_SIZE = 100
NUM_CHECKPOINTS = 15

# Calcular o total de exemplos de treinamento para atingir o STOP_POINT
# Ajusta dinamicamente o número de exemplos de treinamento com base em STOP_POINT
TOTAL_TRAINING_EXAMPLES = int(STOP_POINT * NUM_TRAINING_EXAMPLES)

# Calcula o número de passos por época
steps_per_epoch = TOTAL_TRAINING_EXAMPLES // (BATCH_SIZE * NUM_CHECKPOINTS)


checkpoint_path = './checkpoints/model_checkpoint'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq=steps_per_epoch)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).repeat()

eval_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
eval_dataset = eval_dataset.batch(BATCH_SIZE)

history = model.fit(train_dataset,
                    epochs=NUM_CHECKPOINTS,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=eval_dataset,
                    callbacks=[cp_callback])


test_loss, test_acc = model.evaluate(eval_dataset, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/15
9533/9533 [==============================] - 16s 2ms/step - loss: 1.7677e-08 - accuracy: 1.0000 - val_loss: 0.2059 - val_accuracy: 0.9807
Epoch 2/15
9533/9533 [==============================] - 16s 2ms/step - loss: 7.7600e-09 - accuracy: 1.0000 - val_loss: 0.2100 - val_accuracy: 0.9808
Epoch 3/15
9533/9533 [==============================] - 16s 2ms/step - loss: 5.0331e-09 - accuracy: 1.0000 - val_loss: 0.2129 - val_accuracy: 0.9809
Epoch 4/15
9533/9533 [==============================] - 16s 2ms/step - loss: 3.8166e-09 - accuracy: 1.0000 - val_loss: 0.2150 - val_accuracy: 0.9809
Epoch 5/15
9533/9533 [==============================] - 16s 2ms/step - loss: 3.1356e-09 - accuracy: 1.0000 - val_loss: 0.2171 - val_accuracy: 0.9806
Epoch 6/15
9533/9533 [==============================] - 16s 2ms/step - loss: 2.7284e-09 - accuracy: 1.0000 - val_loss: 0.2187 - val_accuracy: 0.9807
Epoch 7/15
9533/9533 [==============================] - 16s 2ms/step - loss: 2.4934e-09 - accuracy: 1.0000